In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
import os
from keras.models import model_from_json
from skimage.transform import resize
from keras.optimizers import Adam


Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename) 
    if ds.Modality == 'DX' and (int(ds.PatientAge) > 1 and int(ds.PatientAge) < 95) and ds.BodyPartExamined == 'CHEST' and (ds.PatientPosition == 'PA' or ds.PatientPosition == 'AP'):
        return ds.pixel_array
    else:
        print('This study cannot be used with algorithm')
        return None
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    proc_img = resize(img, img_size)
    proc_Img = (proc_img - img_mean)/img_std
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    loaded_json = open(model_path, "r")
    loaded_model = loaded_json.read()
    loaded_json.close()
    model = model_from_json(loaded_model)
    model.load_weights(weight_path)
    
    optimizer = Adam(lr=0.001)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo 
    
    pred = model.predict(img)
    
    for i in range(len(pred)):
        if pred[i] > thresh:
            return  "Pneumonia"
        else:
            return  "No Pneumonia"

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "/home/workspace/my_model_ssheikhAIHCND.json" #path to saved model
weight_path = "/home/workspace/xray_class_my_model.best.hdf5" #path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.58 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
    
    img_mean = np.mean(img) 
    img_std = np.std(img)
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
No Pneumonia
Load file test2.dcm ...
No Pneumonia
Load file test3.dcm ...
No Pneumonia
Load file test4.dcm ...
This study cannot be used with algorithm
Load file test5.dcm ...
This study cannot be used with algorithm
Load file test6.dcm ...
This study cannot be used with algorithm
